### NN

In [16]:
import os
import pandas as pd
import numpy as np
import imageio
import random
import torch
import torch.nn as nn
from sklearn.metrics import roc_curve, roc_auc_score

In [2]:
train_path = './train_data'
df_train = pd.read_csv('./df_train.csv')

val_path = './validation_data'
df_val = pd.read_csv('./df_validation.csv')

small_path = './small_sample'
df_small = pd.read_csv('./df_small.csv')

In [3]:
def read_batch_images(images, df, path):
    features_target = []
    for im in images:
        current_image = np.array(imageio.imread(os.path.join(path, im)))/255
        image_id = int(im.split('.')[0])
        target = int(df[df.id == image_id]['broken'])
        features_target.append((current_image, target))
    return features_target

In [4]:
def batch_generator(df, path, batch_size):
    images_name = [f for f in os.listdir(path) if f.split('.')[1] == 'png']
    random.shuffle(images_name)
    n = len(df)
    for i in range(0, n, batch_size):
        images_filter = images_name[i:(i+batch_size)]
        data = read_batch_images(images_filter, df, path)
        yield data

In [5]:
class BasicNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 16, 3)
        self.conv3 = nn.Conv2d(16, 8, 3)
        self.fc1 = nn.Linear(12800,512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 1)
        self.pool = nn.MaxPool2d(2)
        self.relu = nn.ReLU()
        self.drp1 = nn.Dropout(0.25)
        self.drp2 = nn.Dropout(0.25)
        
    def forward(self, x):
        fo = self.pool(self.relu(self.conv1(x)))
        so = self.pool(self.relu(self.conv2(fo)))
        to = self.pool(self.relu(self.conv3(so))).view(-1,12800)
        fc1_out = self.drp1(self.relu(self.fc1(to)))
        fc2_out = self.drp2(self.relu(self.fc2(fc1_out)))
        out = self.fc3(fc2_out)
        return out

In [12]:
def get_performance_metrics(target, score):
    fpr, tpr, threshold = roc_curve(target, score)
    tnr = [1-f for f in fpr]
    auc_roc = roc_auc_score(target, score)
    df = pd.DataFrame({'sens' : tpr, 'spec': tnr, 'threshold': threshold})
    good_metrics = df[df.sens >= 0.85] 
    index_best = np.argmax(good_metrics.spec)
    df_best = good_metrics.iloc[index_best]
    best_sens = df_best.sens
    best_spec = df_best.spec
    best_th = df_best.threshold

    return  auc_roc, best_sens, best_spec, best_th
    
def train(model, optimizer, criterion, df_train, train_path, df_val, val_path, num_epochs, batch_size, device):
    for i in range(num_epochs):
        total_loss = 0
        val_losses = []
        tr_losses = []
        model.train()
        for batch in batch_generator(df_train, train_path, batch_size):
            x = [d[0] for d in batch]
            y = [d[1] for d in batch]
            n_batch_size = len(x) if len(x) < batch_size else batch_size
            x_t = torch.tensor(x, dtype=torch.float, device=device).view(n_batch_size, 1, 340, 340)
            y_t = torch.tensor(y, dtype=torch.float, device=device)
            model.zero_grad()
            output = model(x_t)
            loss = criterion(output.view(-1), y_t)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        val_probs, ys, val_loss = validate(model, criterion, df_val, val_path, batch_size, device)
        val_losses += [val_loss / df_val.shape[0]]
        tr_losses += [total_loss / df_train.shape[0]]
        probs_val_matrix += [val_probs]
        y_val_matrix += [ys]
        auc_roc, sens, spec, th = get_performance_metrics(ys, val_robs)
        
        print('Epoch {}, avg train loss per image {}, avg valid loss per image {}, auc {}, sens {}, spec {}, th {}'.format(
            i+1, tr_losses[-1], val_losses[-1], auc_roc, sens, spec, th
        ))
    return tr_losses[-1], val_losses[-1]

def validate(model, criterion, df_val, val_path, batch_size, device):
    model.eval()
    val_total_loss = 0
    probs = []
    ys = []
    sigmoid = torch.nn.Sigmoid()
    with torch.no_grad():
        for batch in batch_generator(df_val, val_path, batch_size):
            x = [d[0] for d in batch]
            y = [d[1] for d in batch]
            n_batch_size = len(x) if len(x) < batch_size else batch_size
            x_t = torch.tensor(x, dtype=torch.float, device=device).view(n_batch_size, 1, 340, 340)
            y_t = torch.tensor(y, dtype=torch.float, device=device)
            output = model(x_t)
            probs += list(sigmoid(output).view(-1).detach().cpu().numpy())
            ys += y
            loss = criterion(output.view(-1), y_t)
            val_total_loss += loss.item()
    return probs, ys, val_total_loss

In [13]:
epochs = 10
batch_size = 4
learning_rate = 0.001
device = torch.device('cuda')
model = BasicNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCEWithLogitsLoss(reduction='sum')

In [14]:
train(model=model, optimizer=optimizer, criterion=criterion, 
      df_train=df_small, train_path='./small_sample', df_val=df_small, 
      val_path='./small_sample', num_epochs=epochs, batch_size=batch_size, device=device)

Epoch 1, avg train loss per image 0.6161311364173889, avg valid loss per image 0.5157937049865723
Epoch 2, avg train loss per image 0.42291783332824706, avg valid loss per image 0.2749096381664276
Epoch 3, avg train loss per image 0.2772218157351017, avg valid loss per image 0.1318114759027958
Epoch 4, avg train loss per image 0.04895210769187543, avg valid loss per image 0.011168512779772755
Epoch 5, avg train loss per image 0.9419429136421854, avg valid loss per image 0.03580043733818457
Epoch 6, avg train loss per image 0.15848299771547317, avg valid loss per image 0.14110941261053087
Epoch 7, avg train loss per image 0.08335868597030639, avg valid loss per image 0.015961882639676333
Epoch 8, avg train loss per image 0.09296452790382319, avg valid loss per image 0.03822235038081999
Epoch 9, avg train loss per image 0.06459313649169417, avg valid loss per image 0.00855325518263271
Epoch 10, avg train loss per image 0.012318365257233382, avg valid loss per image 0.011361637858717586


(0.012318365257233382, 0.011361637858717586)